In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import glob
import os

# 루트 경로
base_path = "/content/drive/MyDrive/data/item"

# 모든 직업군 폴더 경로 추출
job_folders = [os.path.join(base_path, folder) for folder in os.listdir(base_path) if os.path.isdir(os.path.join(base_path, folder))]

# 모든 CSV 파일 경로 수집
csv_files = []
for folder in job_folders:
    csv_files.extend(glob.glob(os.path.join(folder, "*.csv")))

# CSV 파일 읽어서 병합
df_list = [pd.read_csv(file) for file in csv_files]
merged_df = pd.concat(df_list, ignore_index=True)

# 결과 확인
print(f"병합된 총 행 수: {merged_df.shape[0]}")
print(f"열 구성: {merged_df.columns.tolist()}")

병합된 총 행 수: 3193715
열 구성: ['nickname', 'world', 'level', 'job', 'subclass', 'preset_number', 'equipment_part', 'equipment_slot', 'item_name', 'str_total', 'dex_total', 'int_total', 'luk_total', 'max_hp_total', 'max_mp_total', 'attack_power_total', 'magic_power_total', 'armor_total', 'speed_total', 'jump_total', 'boss_damage_total', 'ignore_monster_armor_total', 'all_stat_total', 'damage_total', 'equipment_level_decrease_total', 'max_hp_rate_total', 'max_mp_rate_total', 'str_base', 'dex_base', 'int_base', 'luk_base', 'max_hp_base', 'max_mp_base', 'attack_power_base', 'magic_power_base', 'armor_base', 'speed_base', 'jump_base', 'boss_damage_base', 'ignore_monster_armor_base', 'all_stat_base', 'max_hp_rate_base', 'max_mp_rate_base', 'base_equipment_level_base', 'potential_option_grade', 'additional_potential_option_grade', 'potential_option_1', 'potential_option_2', 'potential_option_3', 'additional_potential_option_1', 'additional_potential_option_2', 'additional_potential_option_3', 'equ

In [ ]:
# CSV 저장
save_path = "/content/drive/MyDrive/data/merged/item_merged2.csv" # 저장 경로 수정해주세요
merged_df.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"CSV 저장 완료: {save_path}")

CSV 저장 완료: /content/drive/MyDrive/data/merged/item_merged2.csv


In [ ]:
# item 통합 파일 불러오기

import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/data/merged/item_merged2.csv")

In [ ]:
# 메이플의 장비 프리셋은 총 3개 -> 가장 추가옵션이 높은 프리셋 하나만 사용하여 분석하기

# --- 1. 주스탯 매핑 ---
main_stat_map = {
    "STR": ["히어로", "아델", "소울마스터", "아란", "제로", "팔라딘", "다크나이트", "카이저",
            "데몬슬레이어", "미하일", "블래스터", "은월", "바이퍼", "스트라이커", "캐논마스터", "아크"],
    "DEX": ["윈드브레이커", "메르세데스", "보우마스터", "패스파인더", "신궁", "카인", "와일드헌터",
            "엔젤릭버스터", "캡틴", "메카닉"],
    "INT": ["비숍", "아크메이지(불,독)", "아크메이지(썬,콜)", "라라", "배틀메이지", "에반", "루미너스",
            "키네시스", "플레임위자드", "일리움"],
    "LUK": ["나이트워커", "섀도어", "나이트로드", "듀얼블레이더", "팬텀", "호영", "칼리", "카데나"],
    "HP": ["데몬어벤져"],
    "ALL": ["제논"]  # 제논은 따로 계산 분기
}

# --- 2. 주스탯 판별 함수 ---
def get_main_stat(row):
    for stat, jobs in main_stat_map.items():
        if row["job"] in jobs:
            return stat
    return None

# --- 3. 보너스 추옵 계산 함수 ---
def calculate_bonus_stat(row):
    stat = row["main_stat_type"]
    if stat is None:
        return 0
    allstat = row["all_stat_add"] * 10
    if stat == "INT":
        return row["int_add"] + allstat + row["magic_power_add"] * 4
    elif stat == "HP":
        return row["attack_power_add"] * 150 + row["max_hp_add"] + allstat
    elif stat == "ALL":  # 제논
        return (
            row["str_add"] +
            row["dex_add"] +
            row["luk_add"] +
            row["attack_power_add"] * 4 +
            allstat
        )
    else:  # STR / DEX / LUK
        return row[f"{stat.lower()}_add"] + allstat + row["attack_power_add"] * 4

# --- 4. main_stat_type 및 bonus_stat_total 계산 ---
df["main_stat_type"] = df.apply(get_main_stat, axis=1)
df["bonus_stat_total"] = df.apply(calculate_bonus_stat, axis=1)

# --- 5. preset별 bonus 총합 계산 (nickname + preset_number) ---
preset_scores = (
    df.groupby(["nickname", "preset_number"])["bonus_stat_total"]
    .sum()
    .reset_index(name="preset_score")
)

# --- 6. nickname별로 bonus 총합이 가장 높은 프리셋 선택 ---
best_preset = (
    preset_scores.loc[preset_scores.groupby("nickname")["preset_score"].idxmax()]
    .reset_index(drop=True)
)

# --- 7. 최적 preset의 모든 장비 정보 추출 ---
df_preset = df.merge(
    best_preset[["nickname", "preset_number"]],
    on=["nickname", "preset_number"],
    how="inner"
)

# ✅ 결과 확인
print(df_preset.shape)
print(df_preset['nickname'].nunique(), "명의 유저의 최고 프리셋이 추출됨")
print("예시 유저 장비 수 (24개여야 정상):")
print(df_preset.groupby("nickname")["equipment_slot"].nunique().value_counts())

(1063724, 113)
45938 명의 유저의 최고 프리셋이 추출됨
예시 유저 장비 수 (24개여야 정상):
equipment_slot
24    38589
23     3222
22      787
21      534
20      293
8       250
10      246
9       246
7       226
11      214
19      195
18      157
12      152
13      128
15      127
14      121
6       120
17      116
16      112
5        35
1        22
2        18
4        17
3        11
Name: count, dtype: int64


In [ ]:
# CSV 저장
save_path = "/content/drive/MyDrive/data/preprocessing/item.csv" # 저장 경로 수정해주세요
df_preset.to_csv(save_path, index=False, encoding="utf-8-sig")

print(f"CSV 저장 완료: {save_path}")

CSV 저장 완료: /content/drive/MyDrive/data/preprocessing/item.csv
